In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.optimizers import Adam

#1-спам
#0-не спам
df = pd.read_csv('spam_or_not_spam.csv', delimiter=',')
df.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [ ]:
df.isna().sum()

email    1
label    0
dtype: int64

In [ ]:
df.fillna('hello',inplace=True)
df.isna().sum()

email    0
label    0
dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
y = df['label'].values
x = df['email'].values
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, shuffle = True, random_state=123)
print(x_train.shape)
print(x_test.shape)

(2400,)
(600,)


In [ ]:
import tensorflow as tf
y_train = tf.keras.utils.to_categorical(y_train, num_classes = 2)
y_test = tf.keras.utils.to_categorical(y_test, num_classes = 2)

In [ ]:
print(y_test.shape)

(600, 2)


In [ ]:
tokenizer = Tokenizer(lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(x_train)
word_dict = tokenizer.index_word
print('Кол-во слов:',len(word_dict))
print(word_dict) #От наиболее встречающихся к наименнее

Кол-во слов: 30762
{1: 'number', 2: 'the', 3: 'to', 4: 'of', 5: 'and', 6: 'a', 7: 'in', 8: 'i', 9: 'url', 10: 'is', 11: 'that', 12: 'you', 13: 'it', 14: 'for', 15: 'this', 16: 'on', 17: 's', 18: 'with', 19: 'be', 20: 'have', 21: 'are', 22: 'from', 23: 'not', 24: 'as', 25: 't', 26: 'or', 27: 'your', 28: 'by', 29: 'if', 30: 'at', 31: 'we', 32: 'but', 33: 'can', 34: 'will', 35: 'was', 36: 'an', 37: 'all', 38: 'list', 39: 'my', 40: 'they', 41: 'has', 42: 'so', 43: 'more', 44: 'do', 45: 'one', 46: 'our', 47: 'there', 48: 'get', 49: 'no', 50: 'out', 51: 'just', 52: 'about', 53: 'what', 54: 'which', 55: 'their', 56: 'people', 57: 'only', 58: 'use', 59: 'up', 60: 'would', 61: 'free', 62: 'time', 63: 'email', 64: 'like', 65: 'new', 66: 'any', 67: 'who', 68: 'he', 69: 'some', 70: 'now', 71: 'other', 72: 'don', 73: 'when', 74: 'me', 75: 'here', 76: 'mail', 77: 'been', 78: 'm', 79: 'than', 80: 'how', 81: 'them', 82: 'e', 83: 'also', 84: 'wrote', 85: 'then', 86: 'make', 87: 'mailing', 88: 'message'

Замена слов на цифры

In [ ]:
x_train_s = tokenizer.texts_to_sequences(x_train)
x_test_s = tokenizer.texts_to_sequences(x_test)
print(x_train_s[0])
for el in x_train_s[0]:
    print(word_dict[el], end=' ')
print('\n',x_train[0])

[16, 712, 1, 175, 1, 1881, 4616, 84, 463, 85, 1263, 6, 1145, 4, 1, 1, 1, 1, 1, 655, 176, 3492, 218, 18, 1, 1, 454, 85, 1, 476, 311, 17, 329, 421, 5340, 83, 41, 656, 18, 2, 71, 1, 1, 454, 13, 10, 23, 8239, 3, 161, 2085, 1688, 67, 98, 656, 18, 548, 574, 657, 47, 60, 19, 43, 7, 425, 851, 43, 272, 32, 6, 330, 4, 81, 20, 1586, 72, 25, 20, 3, 19, 6, 16173, 29, 2928, 11, 12, 33, 20, 8240, 18, 548, 657, 7, 6, 570, 179, 7, 6, 16174, 13, 219, 25, 236, 1329, 1284, 3, 44, 1388, 15, 10, 23, 171, 53, 3331, 44, 11906, 23, 808, 95, 535]
on sun number sep number gordon mohr wrote ok then consider a population of number number number number number men each pair off with number number women then number man let s call him wilt also has sex with the other number number women it is not uncommon to find gay males who had sex with several thousands partners there would be more in fact lots more probably but a lot of them have died don t have to be a callboy if considering that you can have intercourse with se

Одинаковая длины (кол-во слов)

In [ ]:
x_train_pad = pad_sequences(x_train_s, maxlen=100)
x_test_pad = pad_sequences(x_test_s, maxlen=100)
print(x_train_pad[:5])

[[  176  3492   218    18     1     1   454    85     1   476   311    17
    329   421  5340    83    41   656    18     2    71     1     1   454
     13    10    23  8239     3   161  2085  1688    67    98   656    18
    548   574   657    47    60    19    43     7   425   851    43   272
     32     6   330     4    81    20  1586    72    25    20     3    19
      6 16173    29  2928    11    12    33    20  8240    18   548   657
      7     6   570   179     7     6 16174    13   219    25   236  1329
   1284     3    44  1388    15    10    23   171    53  3331    44 11906
     23   808    95   535]
 [  162   291    47    10    85     6    92     4   241   784  1552    12
     33   182   133    16  7215   304    69   225    64   451    72    25
   1424     6  7215  1689  2219    12    33    83   422    27   698    22
    133  3673   225    64  9641    67    44  1690  1067     3   182   133
      5    55   162   269     4   133    11    41     6   185  5784    14
     55  16

LSTM

In [ ]:
model = Sequential()
model.add(Embedding(len(word_dict)+1, 128, input_length = 100))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(2, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          3937664   
                                                                 
 lstm (LSTM)                 (None, 100, 128)          131584    
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 4118786 (15.71 MB)
Trainable params: 4118786 (15.71 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Обучение

In [ ]:
history = model.fit(x_train_pad, y_train, epochs=7, batch_size=64, validation_data=(x_test_pad, y_test))

Epoch 1/7
38/38 [==============================] - 25s 506ms/step - loss: 0.6402 - accuracy: 0.8079 - val_loss: 0.5110 - val_accuracy: 0.8517
Epoch 2/7
38/38 [==============================] - 19s 498ms/step - loss: 0.4145 - accuracy: 0.8288 - val_loss: 0.3229 - val_accuracy: 0.8517
Epoch 3/7
38/38 [==============================] - 24s 618ms/step - loss: 0.2682 - accuracy: 0.8517 - val_loss: 0.1596 - val_accuracy: 0.9250
Epoch 4/7
38/38 [==============================] - 27s 723ms/step - loss: 0.1028 - accuracy: 0.9721 - val_loss: 0.0750 - val_accuracy: 0.9817
Epoch 5/7
38/38 [==============================] - 19s 490ms/step - loss: 0.0573 - accuracy: 0.9904 - val_loss: 0.0520 - val_accuracy: 0.9883
Epoch 6/7
38/38 [==============================] - 20s 519ms/step - loss: 0.0334 - accuracy: 0.9933 - val_loss: 0.0403 - val_accuracy: 0.9883
Epoch 7/7
38/38 [==============================] - 19s 489ms/step - loss: 0.0221 - accuracy: 0.9937 - val_loss: 0.0414 - val_accuracy: 0.9867


Тест

In [ ]:
class_name = ['Не спам', 'Спам']
predict_x = model.predict(x_test_pad[:1])
classes_x = np.argmax(predict_x)
print(x_test[:1])
print('Полученное значение:', class_name[classes_x])
print('Ожидаемое значение:', class_name[np.argmax(y_test[:1])])

1/1 [==============================] - 2s 2s/step
[' guido therefore i propose to nuke URL after adding a u feature guido anyone against i imagine that skip or barry might have a stake guido in URL tim seems to have moved all code he s working to guido other modules no argument here skip ']
Полученное значение: Не спам
Ожидаемое значение: Не спам


In [ ]:
predict_x = model.predict(x_test_pad[10:11])
classes_x = np.argmax(predict_x)
print(x_test[10:11])
print('Полученное значение:', class_name[classes_x])
print('Ожидаемое значение:', class_name[np.argmax(y_test[10:11])])

1/1 [==============================] - 0s 74ms/step
[' otc newsletter discover tomorrow s winners for immediate release cal bay stock symbol cbyi watch for analyst strong buy recommendations and several advisory newsletters picking cbyi cbyi has filed to be traded on the otcbb share prices historically increase when companies get listed on this larger trading exchange cbyi is trading around NUMBER cents and should skyrocket to NUMBER NUMBER NUMBER NUMBER a share in the near future put cbyi on your watch list acquire a position today reasons to invest in cbyi a profitable company and is on track to beat all earnings estimates one of the fastest growing distributors in environmental safety equipment instruments excellent management team several exclusive contracts impressive client list including the u s air force anheuser busch chevron refining and mitsubishi heavy industries ge energy environmental research rapidly growing industry industry revenues exceed NUMBER million estimates indi